In [1]:
ORDERS_PATH = '../finals/full_order_all_columns.csv'

In [2]:
import pandas as pd
import numpy as np
class FilesManagement:
    """this class provide operations for files management.""" 
    def read_file(self,**kwargs):
        """read_file(**kwargs)--->reading from given file path, works with pandas and pickle library.
        **kwargs parameters: 
        file_path: is the path of file, required.
        columns_to_rename: path of columns to rename file or dict, optional.
        columns_to_drop: path of columns to drop names file or list, optional, dropping done after rename."""
        try:
            if str(kwargs['file_path']).endswith('.csv'):
                self.df = pd.read_csv(kwargs['file_path'])
            elif str(kwargs['file_path']).endswith('.xslx'):
                self.df = pd.read_csv(kwargs['file_path'])
            else:
                print('not supported file format.')
                raise ValueError()
            self.df = self.__rename_columns__(kwargs.get('columns_to_rename',None))
            self.df = self.__drop_columns__(kwargs.get('columns_to_drop',None))
            return self.df
        except Exception as ex:
            print('Some Error in reading file:',ex)
            raise ex
        return self.df
    
    def __rename_columns__(self,columns):
        try:
            if isinstance(columns,dict):
                return self.df.rename(columns= columns)
            elif isinstance(columns,str):
                
                with open(columns,'rb') as file:
                    columns_to_rename= pickle.load(file)
                return self.df.rename(columns= columns_to_rename)
            else:
                return self.df      
        except Exception as ex:
            print('Some Error in FilesManagement->rename_columns',ex)
            raise ex
            return self.df  
            
    def __drop_columns__(self,columns):
        try:
            if isinstance(columns,list):
                return self.df.drop(columns= columns) 
            elif isinstance(columns,str):
                with open(columns,'rb') as file:
                    columns_to_drop = pickle.load(file)
                return self.df.drop(columns= columns_to_drop)
            else:
                return self.df
        except Exception as ex:
            print('Some Error in FilesManagement->drop_columns',ex)
            return self.df  
        
class BillsAccumelator(FilesManagement):
    """this class provide us to accumelate bills from orders."""
    def __init__(self,**kwargs):
        """init(self,**kwargs)---> 
        **kwargs parameters:
        orders_path: path of orders file,required.
        the orders file must have columns:[bill_id,dish_id]  """
        try:
            self.orders = self.read_file(file_path=kwargs['orders_path'])
            self.groups = self.orders.groupby('bill_id')
            self.bills = {}
        except Exception as ex:
            print('Some Error in BillsAccumelator->init:',ex)
    def accumelate_bills(self,**kwargs):
        """accumelate_bills(**kwargs)---> this function improve us to accumelate bills from orders.
        **kwargs parameters: no parameters"""
        def get_bill_items(bill,bills):
            try:
                bills[bill.name].append(bill['dish_id'].values.tolist())
            except:
                bills[bill.name]= bill['dish_id'].values.tolist()
                
        self.groups.apply(get_bill_items,bills=self.bills)
        bills_ = pd.Series(self.bills)
        self.bills = pd.DataFrame(bills_,columns=['dishes'])
#         self.bills = self.bills.reset_index().rename(columns={'index':'bill_id'})
        return self.bills

In [3]:
import pickle 
class ObjectsManagement:
    """read and write objects on given path.
    works by pickel."""
    def read_object(path):
        try:
            with open(path,'rb') as file:
                object_ = pickle.load(file)
        except Exception as ex:
            print('Some error in reading from file:',path)
            raise ex
        return object_
    
    def write_object(path,obj):
        try:
            with open(path,'wb') as file:
                pickle.dump(obj,file)
        except Exception as ex:
            print('Some error in writing to file:',path)
            raise ex

In [317]:
1/0

ZeroDivisionError: division by zero

In [4]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.mlxtend.frequent_patterns import association_rules
class Recommander:
    """Interface to define rocommanders methods."""
    def prepare_data(self,**kwargs):
        """Transforming Data in way that models accept"""
        pass
    def extract_patterns(self,**kwargs):
        """get patterns from TransactionData"""
        pass
    def build_recommander_model(self, **kwargs):
        """define the model that recommandation system will apply."""
        pass
    def make_recommandation(self,**kwargs):
        """make recommandation on some item."""
        pass

class EvaluationMethods:
    pass

class RulesEvaluation(EvaluationMethods):
    """"""
    def jacard_evalution(self,rule,itemset):
        """jacard_evalution(rule,itemset)--->"""
        try:
            antencedent = rule['antecedents']
            consequent = rule['consequents']
            support_ant = itemset.support[itemset['itemsets'].str.contains(antencedent)].sum()
            support_con = itemset.support[itemset['itemsets'].str.contains(consequent)].sum()
            support_intersection = itemset.support[itemset['itemsets'].str.contains(antencedent) & itemset['itemsets'].str.contains(consequent) ].sum()
            eq = float(support_ant + support_con - support_intersection)
            if eq == 0:
                eq = support_intersection
                
            jaccard = float(support_intersection)/ eq
            ir = np.abs(support_ant - support_con) / eq
            return jaccard, ir 
        
        except Exception as ex:
            print('Some Error in RulesEvaluation->jacard_evalution',ex)
            raise ex
    
class AssosiationRulesRecommander(Recommander):
    """Recommandation System Class Based On AssusiationRules as model,
    and FbGrowth as DataPrepare Algorithm."""
    def __init__(self,**kwargs):
        try:
            self.items = kwargs['items']
            self.evaluation_columns = []
        except Exception as ex:
            print('Some Error in AssosiationRulesRecommander->init',ex)
            raise ex
    def prepare_data(self,**kwargs):
        """prepare_data(**kwargs)---> Transforming Data in way that models accept.
        **kwargs parameters:
        data: the data of Transactions."""
        try:
            te = TransactionEncoder()
            transformed_data = te.fit_transform(self.items)
            self.df = pd.DataFrame(transformed_data, columns=te.columns_)
        except Exception as ex:
            print('Some Error in AssosiationRulesRecommander->prepare_data',ex)
            raise ex
        
    def extract_patterns(self,**kwargs):
        """extract_patterns(**kwargs)---> 
        **kwargs parameters:
        min_support: the minimum support of frequent item to consider as pattern. 
        max_len:"""
        try:
            def frozenset_to_string(x):
                return ','.join(str(i) for i in x)    
            self.df = fpgrowth(self.df, min_support=0.0001,max_len=4)
            self.df_copy = self.df.copy()
            self.df_copy['itemsets'] = self.df_copy['itemsets'].apply(frozenset_to_string)
        except Exception as ex:
            print('Some Error in AssosiationRulesRecommander->build_recommander_model',ex)
            raise ex

    def build_recommander_model(self, **kwargs):
        """"""
        try:
            def frozenset_to_string(x):
                return ','.join(str(i) for i in x)
            
            self.rules = association_rules(self.df, metric="lift", min_threshold=kwargs.get('min_threshold',1000))
            try:
                self.rules['antecedents'] = self.rules.antecedents.apply(frozenset_to_string)
                self.rules['consequents'] = self.rules.consequents.apply(frozenset_to_string)
            except Exception as ex:
                print('Some Error in AssosiationRulesRecommander->transform frozen to string',ex)
        except Exception as ex:
            print('Some Error in AssosiationRulesRecommander->build_recommander_model',ex)
            raise ex 
            
    def evaluation(self,**kwargs):
         try:
            self.evaluation_columns = ['jaccard','ir']
            evaluation_func = kwargs['evaluation_func']
            self.rules[self.evaluation_columns] = self.rules.apply(evaluation_func,itemset=self.df_copy,axis=1,result_type="expand")
            return self.rules
         except Exception as ex:
            print('Some Error in AssosiationRulesRecommander->evaluation',ex)
            raise ex  
    def make_recommandation(self,item_id,**kwargs):
        """make_recommandation(item_id,**kwargs)---> make recommandations for given item id."""
        try:
            reco = self.rules[['antecedents','consequents']+self.evaluation_columns][self.rules['antecedents']==item_id]
            return reco
        except Exception as ex:
            print('Some Error in AssosiationRulesRecommander->make_recommandation',ex)
            raise ex  
        pass

class DishRecommander:
    """this class provide us to give recommandations on the dish"""
    def __init__(self,**kwargs):
        """init(**kwargs)---> 
        **kwagrs parameters: 
        recommander_path: path of trained recommander model, optional.
        """
        try:
            if kwargs.get('recommander_path') != None:
                self.recommander = ObjectsManagement.read_object(kwargs.get('recommander_path'))
        except Exception as ex:
            print('Some Error in DishRecommander->init',ex)
    def fit(self,**kwargs):
        """bulid reccomander and fit data.
        fit(**kwargs)--->
        **kwagrs parameters:
        orders_path: path of orders file must contain bill_id,dish_id field, required."""
        try:
            print('collecting bills...')
            self.ba = BillsAccumelator(orders_path = kwargs['orders_path'])
            self.bills = self.ba.accumelate_bills()
            print('building recommander...')
            self.recommander = AssosiationRulesRecommander(items=self.bills['dishes'])
            self.recommander.prepare_data()
            self.recommander.extract_patterns()
            self.recommander.build_recommander_model()
            ev = RulesEvaluation()
            print('apply evaluation...')
            self.recommander.evaluation(evaluation_func = ev.jacard_evalution)
            print('done.')
        except Exception as ex:
            print('Some Error in DishRecommander->fit',ex) 
            
    def predict(self,item):
        """give you a recommandation."""
        try:
            recommanded_arr = []
            def split_and_merge(item,arr):
                arr += [i for i in item.split(',')]
                
            recommandation = self.recommander.make_recommandation(item)
          
            recommandation.sort_values('jaccard')['consequents'].apply(split_and_merge,arr=recommanded_arr)
            return set(recommanded_arr)
        except Exception as ex:
            print('Some Error in DishRecommander->predict',ex)     

In [334]:
# dr = DishRecommander()

In [335]:
# dr.fit(orders_path = ORDERS_PATH)

collecting bills...
building recommander...
apply evaluation...
done.


In [336]:
dr.predict('11310')

{'11306', '11311', '11314', '11315', '11316', '11317'}

In [5]:
dr1 = DishRecommander(recommander_path= 'association_rules_bill_recommander_1000.pkl')

In [6]:
dr1.predict('11310')

{'11306', '11311', '11314', '11315', '11316', '11317'}